In [4]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as T
from tqdm import tqdm
import argparse
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
from vpr_model import VPRModel
import faiss
import faiss.contrib.torch_utils
from prettytable import PrettyTable
# Dataloader
from dataloaders.val.NordlandDataset import NordlandDataset
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os
# from dataloaders.val.MapillaryDataset import MSLS
# from dataloaders.val.MapillaryTestDataset import MSLSTest
# from dataloaders.val.PittsburghDataset import PittsburghDataset
# from dataloaders.val.SPEDDataset import SPEDDataset

# VAL_DATASETS = ['MSLS', 'MSLS_Test', 'pitts30k_test', 'pitts250k_test', 'Nordland', 'SPED']
VAL_DATASETS = ['Nordland']


def input_transform(image_size=None):
    MEAN=[0.485, 0.456, 0.406]; STD=[0.229, 0.224, 0.225]
    if image_size:
        return T.Compose([
            T.Resize(image_size,  interpolation=T.InterpolationMode.BILINEAR),
            T.ToTensor(),
            T.Normalize(mean=MEAN, std=STD)
        ])
    else:
        return T.Compose([
            T.ToTensor(),
            T.Normalize(mean=MEAN, std=STD)
        ])

def get_val_dataset(dataset_name, image_size=None):
    dataset_name = dataset_name.lower()
    transform = input_transform(image_size=image_size)
    
    if 'nordland' in dataset_name:    
        ds = NordlandDataset(input_transform=transform)
    # elif 'msls_test' in dataset_name:
    #     ds = MSLSTest(input_transform=transform)

    # elif 'msls' in dataset_name:
    #     ds = MSLS(input_transform=transform)

    # elif 'pitts' in dataset_name:
    #     ds = PittsburghDataset(which_ds=dataset_name, input_transform=transform)

    # elif 'sped' in dataset_name:
    #     ds = SPEDDataset(input_transform=transform)
    else:
        raise ValueError
    
    num_references = ds.num_references
    num_queries = ds.num_queries
    ground_truth = ds.ground_truth
    # print( "==================  label of some place ==================== ")  
    # print( ground_truth[:5][1] ) 
    return ds, num_references, num_queries, ground_truth

def get_descriptors(model, dataloader, device):
    descriptors = []
    with torch.no_grad():
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            for batch in tqdm(dataloader, 'Calculating descritptors...'):
                imgs, labels = batch
                output = model(imgs.to(device)).cpu()
                descriptors.append(output)

    return torch.cat(descriptors)

def denormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    """
    Giúp khôi phục ảnh từ dạng normalized về dạng hiển thị được (0-1 range).
    """
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)  # t = t * std + mean
    return torch.clamp(tensor, 0, 1)
    
def load_model(ckpt_path):
    model = VPRModel(
        backbone_arch='dinov2_vitb14',
        backbone_config={
            'num_trainable_blocks': 4,
            'return_token': True,
            'norm_layer': True,
        },
        agg_arch='SALAD',
        agg_config={
            'num_channels': 768,
            'num_clusters': 64,
            'cluster_dim': 128,
            'token_dim': 256,
        },
    )

    # Load checkpoint
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    
    # Lấy đúng phần state_dict bên trong
    model.load_state_dict(checkpoint['state_dict'], strict=False)

    model = model.eval()
    model = model.to('cuda')
    print(f"Loaded model from {ckpt_path} successfully!")
    return model

def parse_args():
    parser = argparse.ArgumentParser(
        description="Eval VPR model",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    # Model parameters
    parser.add_argument("--ckpt_path", type=str, required=True, default=None, help="Path to the checkpoint")
    
    # Datasets parameters
    parser.add_argument(
        '--val_datasets',
        nargs='+',
        default=VAL_DATASETS,
        help='Validation datasets to use',
        choices=VAL_DATASETS,
    )
    parser.add_argument('--image_size', nargs='*', default=None, help='Image size (int, tuple or None)')
    parser.add_argument('--batch_size', type=int, default=512, help='Batch size')

    args = parser.parse_args()

    # Parse image size
    if args.image_size:
        if len(args.image_size) == 1:
            args.image_size = (args.image_size[0], args.image_size[0])
        elif len(args.image_size) == 2:
            args.image_size = tuple(args.image_size)
        else:
            raise ValueError('Invalid image size, must be int, tuple or None')
        
        args.image_size = tuple(map(int, args.image_size))

    return args


Using cache found in /home/jupyter-iec_23se07/.cache/torch/hub/facebookresearch_dinov2_main
/tmp/ipykernel_352259/1008000905.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.78 GiB of which 17.50 MiB is free. Process 94290 has 1.53 GiB memory in use. Process 338791 has 4.15 GiB memory in use. Process 347769 has 1.91 GiB memory in use. Including non-PyTorch memory, this process has 172.00 MiB memory in use. Of the allocated memory 38.07 MiB is allocated by PyTorch, and 3.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

torch.backends.cudnn.benchmark = True

model = load_model('./logs/lightning_logs/version_60/checkpoints/last.ckpt')

val_dataset, num_references, num_queries, ground_truth = get_val_dataset('Nordland', [126, 126] )
val_loader = DataLoader('Nordland', num_workers=16, batch_size=16, shuffle=False, pin_memory=True)

    print(f'Evaluating on {val_name}')
    descriptors = get_descriptors(model, val_loader, 'cuda')
    
    print(f'Descriptor dimension {descriptors.shape[1]}')
    r_list = descriptors[ : num_references]

    print('total_size', descriptors.shape[0], num_queries + num_references) 

    # descriptors dimension is 8448 
    # testing = True # isinstance(val_dataset, MSLSTest)

    preds = get_validation_recalls(
        r_list=r_list,
        q_list=q_list,
        k_values=[1, 5, 10, 15, 20, 25],
        gt=ground_truth,
        print_results=True,
        dataset_name=val_name,
        faiss_gpu=False,
        testing=False,
    )

    embed_size = r_list.shape[1]
        if faiss_gpu:
            res = faiss.StandardGpuResources()
            flat_config = faiss.GpuIndexFlatConfig()
            flat_config.useFloat16 = True
            flat_config.device = 0
            faiss_index = faiss.GpuIndexFlatL2(res, embed_size, flat_config)
        # build index
        else:
            faiss_index = faiss.IndexFlatL2(embed_size)

        # add references
        faiss_index.add(r_list)

        # search for queries in the index
        _, predictions = faiss_index.search(q_list, max(k_values))

    # Chọn 2 query để hiển thị
    
    # for q_idx in [1000, 1]:
    #     print(f"\nQuery index: {q_idx}")
    #     pred_ids = preds[q_idx][:4]  # top-4 predictions
    
    #     # Lấy ảnh query và reference từ dataset (tensor), rồi denormalize để hiển thị
    #     query_img, _ = val_dataset[num_references + q_idx]
    #     query_img = denormalize(query_img.clone())
    #     query_img_np = TF.to_pil_image(query_img)
    
    #     fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    #     axes[0].imshow(query_img_np)
    #     axes[0].set_title("Query")
    #     axes[0].axis('off')
    
    #     for i, pred_id in enumerate(pred_ids):
    #         ref_img, _ = val_dataset[pred_id]
    #         ref_img = denormalize(ref_img.clone())
    #         ref_img_np = TF.to_pil_image(ref_img)
    
    #         axes[i+1].imshow(ref_img_np)
    #         axes[i+1].set_title(f"Top-{i+1}")
    #         axes[i+1].axis('off')
    
    #     plt.tight_layout()
    #     plt.savefig(f"./query_{q_idx}_top4.png")
    #     plt.close()


    # print( preds ) 
    
    # if testing:
    #     val_dataset.save_predictions(preds, '../preds.txt')

    del descriptors
    print('========> DONE!\n\n')